# More climatology reductions using Cubed

This is the Cubed equivalent of [More climatology reductions](climatology-hourly.ipynb).

The task is to compute an hourly climatology from an hourly dataset with 744 hours in each chunk, using the "map-reduce" strategy.

In [ ]:
import cubed
import cubed.array_api as xp
import numpy as np
import pandas as pd
import xarray as xr

import flox.xarray

## Create data

Note that we use fewer lat/long points so the computation can be run locally.

In [ ]:
spec = cubed.Spec(allowed_mem="2GB")
ds = xr.Dataset(
    {
        "tp": (
            ("time", "latitude", "longitude"),
            xp.ones((8760, 72, 144), chunks=(744, 5, 144), dtype=np.float32, spec=spec),
        )
    },
    coords={"time": pd.date_range("2021-01-01", "2021-12-31 23:59", freq="h")},
)
ds

## Computation

In [ ]:
hourly = flox.xarray.xarray_reduce(ds.tp, ds.time.dt.hour, func="mean", reindex=True)
hourly

In [ ]:
hourly.compute()

## Other climatologies: resampling by month

This uses the "blockwise" strategy.

In [ ]:
monthly = ds.tp.resample(time="ME").sum(method="blockwise")
monthly

In [ ]:
monthly.compute()